### Interacting with the SHEEP server

Assuming the sheep server is running, and the ```sheep_client.py``` module has the correct BASE_URL, we can configure and run SHEEP jobs using the commands in this notebook.

First lets just set up a directory path (we only need this so we can find the circuit file later on):

In [1]:
import os
if "SHEEP_HOME" in os.environ.keys():
  SHEEP_HOME = os.environ["SHEEP_HOME"]
else:
  SHEEP_HOME = os.path.join(os.environ["HOME"],"SHEEP","frontend")
import sys
sys.path.append(SHEEP_HOME)

In [2]:
from pysheep import sheep_client
         

DB LOCATION IS /Users/nbarlow/SHEEP/frontend/pysheep/sheep.db


In [3]:
sheep_client.new_job()

{'content': '', 'status_code': 200}

List the available contexts:

In [4]:
sheep_client.get_available_contexts()

{'content': ['Clear', 'HElib_F2', 'HElib_Fp'], 'status_code': 200}

Lets do a test with HElib_F2:

In [5]:
sheep_client.set_context("HElib_F2")

{'content': '', 'status_code': 200}

List the available input types:

In [6]:
sheep_client.get_available_input_types()

{'content': ['bool',
  'uint8_t',
  'uint16_t',
  'uint32_t',
  'int8_t',
  'int16_t',
  'int32_t'],
 'status_code': 200}

Let's use uint16_t:

In [7]:
sheep_client.set_input_type("uint16_t")

{'content': '', 'status_code': 200}

Now let's give it a circuit.  There are plenty of simple circuits in the ```benchmark_inputs/``` directory.  We need to provide the full path to the circuit though so we'll use the SHEEP_HOME we defined at the start.

In [8]:
r=sheep_client.set_circuit(os.path.join(SHEEP_HOME,"benchmark_inputs/low_level/circuits/circuit-ADD-1.sheep"))

We can see the names of the "inputs" that this circuit expects:

In [9]:
sheep_client.get_inputs()

{'content': ['input_0', 'input_1'], 'status_code': 200}

Now we can set values for those inputs.  The circuit we've chosen just adds the two inputs together, so lets do 55+44 (and hope we get the answer 99 at the end!)  Note that we have to write the inputs as lists, as potentially we can do more than one calculation at the same time (known as SIMD)).

In [10]:
sheep_client.set_inputs({"input_0":[55],"input_1":[44]})

{'content': '', 'status_code': 200}

We have set the context, the input_type, the circuit_file and the input values - we should be ready to go!  Let's print out the configuration, and then check that we are fully configured.

In [11]:
sheep_client.get_config()

{'content': {'circuit_filename': '',
  'context': 'HElib_F2',
  'eval_strategy': 'serial',
  'input_type': 'uint16_t',
  'nslots': 4,
  'parameters': {'BaseParamSet': 0,
   'BitsPerLevel': 25,
   'Bootstrap': 1,
   'HammingWeight': 128,
   'Levels': 30,
   'c': 2,
   'd': 12,
   'g1': 71,
   'g2': 76,
   'g3': 0,
   'm': 105,
   'm1': 3,
   'm2': 35,
   'm3': 0,
   'ord1': 2,
   'ord2': 2,
   'ord3': 0,
   'phim': 48}},
 'status_code': 200}

In [12]:
sheep_client.is_configured()

{'content': {'job_configured': True}, 'status_code': 200}

OK, let's execute the test!

In [13]:
sheep_client.run_job()

{'content': '', 'status_code': 200}

The SHEEP server just used our selected "context" (i.e. HE library) to encrypt our inputs, evaluate our circuit, and decrypt the outputs.  It also then evaluated the same circuit "in the clear" (i.e. with no encryption) so that we can check we got the right answer.   The result, including the output values, the processing times, and the clear-text check, are available as a json object.

In [14]:
r=sheep_client.get_results()
from pprint import pprint
pprint(r)

{'content': {'cleartext check': {'is_correct': True},
             'outputs': {'output_0': ['99']},
             'timings': {'decryption': '5124.501000',
                         'encryption': '12725.681000',
                         'evaluation': '1316832.731000'}},
 'status_code': 200}


So we can see that 
 * we got the answer "99" (which is 55+44, the sum of our two inputs) 
 * all the output values we got (just one in this case) matched the result from the "cleartext" context.
 * from the timings (which are in microseconds), the encryption and decryption were fast, but evaluating the circuit took some time.